In [4]:
# %%writefile alpha_base10.py

import sys
import os 
module_path = os.path.abspath(os.path.join('..')) 
if module_path not in sys.path: 
    sys.path.append(module_path)
    
import numpy as np
import pandas as pd
from scipy import stats
from sklearn import preprocessing
from sklearn import linear_model
import tools.Pretreat_Tools as pretreat


def STD(data, windows):
    return data.rolling(window=windows, min_periods=windows).std()
def MEAN(data, windows):
    return data.rolling(window=windows, min_periods=windows).mean()
def DELTA(data, windows):
    return data.diff(4)
def SEQUENCE(n):
    return pd.Series(np.arange(1,n+1))

def SMA(data,windows,alpha):
    return data.ewm(adjust=False, alpha=float(alpha)/windows, min_periods=windows, ignore_na=False).mean()

def REGBETA(xs, y, n):
    assert len(y)>=n,  'len(y)!>=n !!!'+ str(y.index[0])
    regress = linear_model.LinearRegression(fit_intercept=False)
    def reg(X,Y):
        try:
            if len(Y)>len(X):
                Y_ =  Y[X.index]
                if Y_.isnull().any():
                    return np.nan
                res = regress.fit(X.values.reshape(-1, 1), Y_.values.reshape(-1, 1)).coef_[0]
            else:
                # if Y.isnull().any():
                #     return np.nan
                res = regress.fit(X.values.reshape(-1, 1), Y.values.reshape(-1, 1)).coef_[0]
        except Exception as e:
            print(e)
            return np.nan
        return res
    return xs.rolling(window=n, min_periods=n).apply(lambda x:reg(x,y))


def COVIANCE(A,B,d):
    se = pd.Series(np.arange(len(A.index)),index=A.index)
    se = se.rolling(5).apply(lambda x: A.iloc[x].cov(B.iloc[x]))
    return se

def CORR(A,B,d):
    se = pd.Series(np.arange(len(A.index)),index=A.index)
    se = se.rolling(5).apply(lambda x: A.iloc[x].corr(B.iloc[x]))
    return se

def excute_for_multidates(data, func, level=0, **pramas):
    return data.groupby(level=level, group_keys=False).apply(func,**pramas)



In [31]:
from tools.Cacher import (CACHE_TYPE, load_cache,load_caches_adv)
import tools.Sample_Tools as smpl
import tools.Pretreat_Tools as pretreat

%load_ext autoreload
%autoreload 2
%aimport tools.Cacher

stock_df = load_cache('all_train_qfq',cache_type=CACHE_TYPE.STOCK).sort_index()
# stock_df = pd.concat(list(map(lambda file:load_cache(file,cache_type=CACHE_TYPE.STOCK),['all_train_qfq','all_tail_qfq','all_older_qfq']))).sort_index()
ret_fs = pd.read_csv(module_path+'/data/static/china10yearbond.csv').set_index('date').sort_index()
ret_fs = (ret_fs['high']+ret_fs['low'])/2 * 0.01
ret_fs_daily = ret_fs/252
ret_fs_daily

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


date
2002-07    0.000114
2002-08    0.000113
2002-09    0.000122
2002-10    0.000122
2002-11    0.000132
             ...   
2022-09    0.000108
2022-10    0.000109
2022-11    0.000111
2022-12    0.000116
2023-01    0.000116
Length: 247, dtype: float64

In [73]:
# codes = smpl.get_codes_by_market(sse='all',only_main=True,filter_st=True)
# 
# test = stock_df.loc[(stock_df.index.get_level_values(0).unique()[0:3],slice(None)),:]
# ret_f = ret_fs_daily.get(test.index[0][0].strftime('%Y-%m'), default=ret_fs_daily[-1])
# ret_t = smpl.get_current_return(stock_df,'close')
# ret_t_excess = ret_t.groupby(pd.Grouper(level='date', freq='1M')).apply(lambda x:x-ret_fs_daily.get(x.index[0][0].strftime('%Y-%m')))

def cal_ret_markey_daily(stock_daily_df,ret_t_excess_daily):
    ##  不取对数有时候有精度问题,权重不是精确1
    market_value_log = np.log(stock_daily_df['totalCapital'])
    market_value = market_value_log.sum()
    weight = market_value_log / market_value
    ret_market_f = (ret_t_excess_daily * weight).sum()
    return ret_market_f

ret_market_t = ret_t_excess.groupby(level=0, group_keys=False).apply(lambda ret: cal_ret_markey_daily(stock_df.loc[ret.index[0][0]],ret))

window=252
half_life_window = 63
half_life = list(map(lambda n:0.5**(n/half_life_window),range(1,window)))
half_life_weight = half_life/np.sum(half_life)


def beta():
    pass

# ret_t.loc[('2017-01',slice(None))]
# data.groupby(level=level, group_keys=False).apply(func,**pramas)



model = linear_model.LinearRegression(fit_intercept=True).fit(X, factor,sample_weight=half_life_weight)
# neutralize_factor = factor - model.predict(X)

In [120]:
ret_t_excess.index#.loc[(slice(None),'000001')].iloc[1:252]

MultiIndex([('2017-01-03', '000001'),
            ('2017-01-03', '000002'),
            ('2017-01-03', '000006'),
            ('2017-01-03', '000007'),
            ('2017-01-03', '000008'),
            ('2017-01-03', '000009'),
            ('2017-01-03', '000010'),
            ('2017-01-03', '000011'),
            ('2017-01-03', '000012'),
            ('2017-01-03', '000014'),
            ...
            ('2021-12-31', '605507'),
            ('2021-12-31', '605555'),
            ('2021-12-31', '605566'),
            ('2021-12-31', '605567'),
            ('2021-12-31', '605577'),
            ('2021-12-31', '605580'),
            ('2021-12-31', '605588'),
            ('2021-12-31', '605589'),
            ('2021-12-31', '605598'),
            ('2021-12-31', '605599')],
           names=['date', 'code'], length=3247889)

In [103]:
ret_market_t.iloc[1:252]

date
2017-01-04    0.011551
2017-01-05    0.001280
2017-01-06   -0.006829
2017-01-09    0.006490
2017-01-10   -0.002299
                ...   
2018-01-05    0.001047
2018-01-08    0.003275
2018-01-09    0.000214
2018-01-10   -0.008277
2018-01-11    0.005182
Name: ret, Length: 251, dtype: float64

In [130]:
# model = linear_model.LinearRegression(fit_intercept=True).fit(ret_t_excess.loc[(slice(None),'000006')].iloc[1:252].values.reshape(-1, 1), ret_market_t.iloc[1:252].values.reshape(-1, 1),sample_weight=half_life_weight)
# model.coef_,model.intercept_
# x = linear_model.LinearRegression(fit_intercept=True)
model = x.fit(ret_t_excess.loc[(slice(None),'000001')].iloc[1:252].values.reshape(-1, 1), ret_market_t.iloc[1:252].values.reshape(-1, 1),sample_weight=half_life_weight)
model.coef_,model.intercept_
alpha_camp_tmp
beta_camp_tmp
# (array([[0.1484555]]), array([-0.00037754]))
ret_t_excess.loc[(slice(None),'000001')].rolling(window).

(array([[0.1484555]]), array([-0.00037754]))